In [1]:
import os
from glob import glob
import numpy as np
import dask
import xarray as xr
import xgcm
from xgcm.autogenerate import generate_grid_ds
from functools import partial
from scipy.signal import butter, filtfilt
from cmocean import cm

from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

In [2]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster()
w = cluster.scale(14)

In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://10.148.0.255:49166 Dashboard: http://10.148.0.255:8787/status,Cluster Workers: 14 Cores: 14 Memory: 239.96 GB


_____________
# 2. read w'b'

In [5]:
out_dir_zarr = '/home/c11-data/Test_aponte/xiaolong/Parameter/data_rechunk/temporary/'
wb_prime_filter = xr.open_zarr(out_dir_zarr+'wb_prime_1day_filter.zarr')
wb_prime_filter

<xarray.Dataset>
Dimensions:          (depth: 52, time: 9415)
Coordinates:
  * depth            (depth) float64 0.5 1.57 2.79 4.185 ... 900.1 945.6 992.3
  * time             (time) float64 0.0 0.04167 0.08333 ... 392.2 392.2 392.2
Data variables:
    wb_prime_filter  (time, depth) float32 dask.array<chunksize=(9415, 1), meta=np.ndarray>

In [7]:
# Coarse-grained PI (spatial average and cumulative integral)
out_dir_store = '/home/c11-data/Test_aponte/xiaolong/Parameter/data_rechunk/temporary/cg_butter/'
PI_cg = xr.open_zarr(out_dir_store+'PI_cg_butter_5d_all.zarr')
PI_cg_smean = PI_cg.PI_cg_smean
PI_cg_smean

<xarray.DataArray 'PI_cg_smean' (depth: 52, time: 9415)>
dask.array<zarr, shape=(52, 9415), dtype=float32, chunksize=(1, 9415), chunktype=numpy.ndarray>
Coordinates:
  * depth    (depth) float64 0.5 1.57 2.79 4.185 ... 855.8 900.1 945.6 992.3
  * time     (time) float64 0.0 0.04167 0.08333 0.125 ... 392.2 392.2 392.2

In [8]:
ds = xr.merge([PI_cg_smean,wb_prime_filter])
# define (real) time
def iters_to_date(iters, delta_t=3600.):
    t0 = datetime.datetime(2011,9,13,0)    
    ltime = delta_t * (np.array(iters))
    dtime = [t0+dateutil.relativedelta.relativedelta(seconds=t) for t in ltime]    
    return dtime

time_day = iters_to_date(np.arange(9415))
ds = ds.assign_coords(time=time_day) 
#ds.persist()
ds

<xarray.Dataset>
Dimensions:          (depth: 52, time: 9415)
Coordinates:
  * depth            (depth) float64 0.5 1.57 2.79 4.185 ... 900.1 945.6 992.3
  * time             (time) datetime64[ns] 2011-09-13 ... 2012-10-09T06:00:00
Data variables:
    PI_cg_smean      (depth, time) float32 dask.array<chunksize=(1, 9415), meta=np.ndarray>
    wb_prime_filter  (time, depth) float32 dask.array<chunksize=(9415, 1), meta=np.ndarray>

In [10]:
winter = ds.isel(depth=slice(16,39)).sel(time=slice('2011-12-01T00:00:00','2012-03-01T00:00:00'))
winter

<xarray.Dataset>
Dimensions:          (depth: 23, time: 2185)
Coordinates:
  * depth            (depth) float64 52.69 60.28 68.69 ... 410.4 441.3 473.3
  * time             (time) datetime64[ns] 2011-12-01 ... 2012-03-01
Data variables:
    PI_cg_smean      (depth, time) float32 dask.array<chunksize=(1, 2185), meta=np.ndarray>
    wb_prime_filter  (time, depth) float32 dask.array<chunksize=(2185, 1), meta=np.ndarray>

In [13]:
# save
file_out = out_dir_zarr+'/wb_PI_winter_1016.nc'
%time winter.to_netcdf(file_out, mode='w')   

CPU times: user 252 ms, sys: 36 ms, total: 288 ms
Wall time: 2.8 s


In [14]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
